In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-08 20:09:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-05-08 20:09:17 (6.45 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon_reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [8]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"*":"count"}).withColumnRenamed("count(1)", "customer_count")
customers_df.count()

4056329

In [9]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0016P392O|iMicro SP-IM942 M...|
|B00PPRYEZA|CoulVue Holder an...|
|B00VZXRRGG|Hexagon Network -...|
|B007GF9N96|Computer USB-Powe...|
|B00LCK3TEG|           Fire HD 6|
|B00KMOG5FS|Fintie 10.1" Tabl...|
|B00CFL5CRI|Cable Matters 2-P...|
|B008NP2AUC|USB USB-2, USB-3 ...|
|B00OAJ5N6I|Samsung 850 EVO -...|
|B00H02XN2I|Techno Earth Rubb...|
|B00YSC6UW8|5 Mm Thick Office...|
|B00CRXZPC6|Lenovo Ideatab S6...|
|B009SKTZFY|Bosign Tabletpill...|
|B003NZSX7G|Netgear N300 WiFi...|
|B0089JIDLC|Kingston ValueRAM...|
|B001E45XT4|Plantronics GameC...|
|B002IY4V9C|Swiss Gear Men's ...|
|B000UVRU6G|SteelSeries QcK G...|
|B00X5X3LQY|EVGA Pro SLI Brid...|
|B00L49X8E6|ASUS X551 15.6-in...|
+----------+--------------------+
only showing top 20 rows



In [13]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3ARRMDEGED8RD|   22873041|B00KJWQIIC|     335625766| 2015-08-31|
| RQ28TSA020Y6J|   30088427|B013ALA9LA|     671157305| 2015-08-31|
| RUXJRZCT6953M|   20329786|B00PML2GQ8|     982036237| 2015-08-31|
| R7EO0UO6BPB71|   14215710|B001NS0OZ4|     576587596| 2015-08-31|
|R39NJY2YJ1JFSV|   38264512|B00AQMTND2|     964759214| 2015-08-31|
|R31SR7REWNX7CF|   30548466|B00KX4TORI|     170101802| 2015-08-31|
| RVBP8I1R0CTZ8|     589298|B00P17WEMY|     206124740| 2015-08-31|
|R1QF6RS1PDLU18|   49329488|B00TR05L9Y|     778403103| 2015-08-31|
|R23AICGEDAJQL1|   50728290|B0098Y77OG|     177098042| 2015-08-31|
|R2EY3N4K9W19UP|   37802374|B00IFYEYXC|     602496520| 2015-08-31|
| RC9AW4HKJ016M|   52027882|B0091ITP0S|     977217357| 2015-08-31|
|R2ALWJE9N6ZBXD|   41770239|B008I21EA2|     295632907| 2015-08

In [14]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3ARRMDEGED8RD|          5|            0|          0|   N|                Y|
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
| RUXJRZCT6953M|          1|            2|          2|   N|                N|
| R7EO0UO6BPB71|          1|            0|          0|   N|                Y|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|                Y|
|R31SR7REWNX7CF|          5|            0|          0|   N|                Y|
| RVBP8I1R0CTZ8|          3|            1|          2|   N|                Y|
|R1QF6RS1PDLU18|          4|            1|          1|   N|                Y|
|R23AICGEDAJQL1|          1|            0|          0|   N|                Y|
|R2EY3N4K9W19UP|          5|            3|          4|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [17]:
# Configure settings for RDS
from getpass import getpass
password = getpass('Enter database password')
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cp9vtji419xj.us-east-2.rds.amazonaws.com:5432/amazon_reviews"
config = {"user":"postgres", 
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [23]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [29]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [27]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [26]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)